In [1]:
import numpy as np
import pybullet as p
import pybullet_data
import matplotlib.pyplot as plt

import time

from utils import *
from casadi import *
import crocoddyl
from costs import *
from ocp_sys import QuadcopterCasadi, ActionDataRobot, ActionModelRobot
from IPython.display import clear_output
from ocp import ILQR_Standard, get_ilqr_from_ddp
from mixture_model import Gaussian
np.set_printoptions(precision=3, suppress=True)
%load_ext autoreload
%autoreload 2

### Configure pybullet

In [2]:
# configure pybullet and load plane.urdf and quadcopter.urdf
physicsClient = p.connect(p.GUI, options='--background_color_red=.6 --background_color_green=.6 --background_color_blue=.6')
p.configureDebugVisualizer(p.COV_ENABLE_GUI,0)
p.setAdditionalSearchPath(pybullet_data.getDataPath())

In [30]:
p.resetSimulation()

#### Load robot model

In [31]:
color_path = [1,1,1,.4]
color_ref_cond = [.95,.95,0,.5]
color_ref_mean = [0,.9,.9,.5]
color_robot = [1,.3,0,1]
color_obstacle = [.1,.1,.1,.9]
color_target = [1,1,1,0.]

In [32]:
robotId = p.loadURDF("../files/quadrotor.urdf",[0,0,1],p.getQuaternionFromEuler([0,0,0]))

for i in range((p.getNumJoints(robotId))+1):
    p.changeVisualShape(robotId, i-1, rgbaColor=[1,0,0,0.6])

_,_,target_id = create_primitives(p.GEOM_BOX, halfExtents=[0.1, 0.1, 0.1], rgbaColor=color_target)
_,_,waypoint_id = create_primitives(p.GEOM_BOX, halfExtents=[0.1, 0.1, 0.1], rgbaColor=color_target)
_,_,waypoint_id2 = create_primitives(p.GEOM_BOX, halfExtents=[0.1, 0.1, 0.1], rgbaColor=color_target)

#### Load obstacle

#### Load multiple spheres

In [33]:
pos_obs_set = [np.array([0, 2., 2.]),np.array([2., 2., 2.]), np.array([-2., 2., 2.]), np.array([1, 4., 2.]),  np.array([-1., 4., 2.]), ] 
#pos_obs_set = [np.array([0, 2., 2.]),np.array([2, 2., 2.]), np.array([-2, 2., 2.])]#, np.array([1, 4., 2.]),  np.array([-1, 4., 2.]), ] 
r_body = 0.35
r_obs = 0.4
margin = 0.1

w_obstacle = 1000

ballIds = []
cost_collisions = []
for pos_obs in pos_obs_set:
    _,_,ballid = create_primitives(radius = r_obs)
    p.resetBasePositionAndOrientation(ballid, pos_obs, (0,0,0,1))
    ballIds += [ballid]
    col_activation = ActivationCollision(nr = 3, threshold = r_body + r_obs + margin)
    cost_collision_i = SphereSphereCollisionCost(col_activation, nu = 4, r_body = r_body, r_obs = r_obs, pos_obs = pos_obs, w = w_obstacle)
    cost_collisions += [cost_collision_i]

#### Capsule obstacles

#### Define the robot_sys

In [34]:
from ocp_sys import Bicopter

In [35]:
sys_name = 'quadcopter' #the choices: ['quadcopter', 'bicopter', 'pendulum']

use_collision = True

if sys_name == 'quadcopter':
    T = 150
    dt = 0.05
    robot_sys = QuadcopterCasadi(dt = dt)
    T_short = 30
    n_iter_short = 5
elif sys_name == 'bicopter':
    T = 150
    dt = 0.05
    T_short = 30
    n_iter_short = 10
    robot_sys = Bicopter(dt = dt)

In [36]:
cost_collision = CostModelSum(robot_sys, cost_collisions)

In [37]:
#### Load multiple robots for visualization
robotIds = []
for t in range(T):
    _,_,rob_id = create_primitives(radius = 0.05)
    #robotIds += [p.loadURDF("../files/quadrotor.urdf",[0,0,1],p.getQuaternionFromEuler([0,0,0]))]
    robotIds += [rob_id]
    p.changeVisualShape(rob_id, -1, rgbaColor=color_path)
        

In [38]:
# for visualizing the reference
robotIdRefs = []
for t in range(T_short):
    #robotIdRefs += [p.loadURDF("../files/quadrotor.urdf",[0,0,1],p.getQuaternionFromEuler([0,0,0]))]
    _,_,rob_id = create_primitives(radius = 0.05)
    robotIdRefs += [rob_id]

    for i in range((p.getNumJoints(robotIdRefs[-1]))+1):
        p.changeVisualShape(robotIdRefs[-1], i-1, rgbaColor=color_ref_cond)
        

In [39]:
# for visualizing the reference
robotIdRefMeans = []
for t in range(T_short):
#    robotIdRefMeans += [p.loadURDF("../files/quadrotor.urdf",[0,0,1],p.getQuaternionFromEuler([0,0,0]))]
    _,_,rob_id = create_primitives(radius = 0.05)
    robotIdRefMeans += [rob_id]
    for i in range((p.getNumJoints(robotIdRefMeans[-1]))+1):
        p.changeVisualShape(robotIdRefMeans[-1], i-1, rgbaColor=color_ref_mean)

# 1. Plan using iLQR with long horizon

#### Setting cost

In [100]:
if sys_name == 'quadcopter':
    x_waypoint = np.array([-1.,2.,2.,0,0,0,0,0,0,0,0,0])
    T_waypoint = 60
    x_waypoint2 = np.array([0,4.,2.,0,0,0,0,0,0,0,0,0])
    T_waypoint2 = 100
    x_ref = np.array([0,7.,2.,0,0,0,0,0,0,0,0,0]) #for quadcopter
    p.resetBasePositionAndOrientation(target_id, x_ref[:3], (0,0,0,1))
    p.resetBasePositionAndOrientation(waypoint_id, x_waypoint[:3], (0,0,0,1))
    p.resetBasePositionAndOrientation(waypoint_id2, x_waypoint2[:3], (0,0,0,1))
    Q = np.eye(robot_sys.Dx)*.1
    Qf = np.eye(robot_sys.Dx)*1000
    R = np.eye(robot_sys.Du)*.1
elif sys_name == 'bicopter':
    x_ref = np.array([3,3,0,0,0,0]) #for bicopter
    Q = np.eye(robot_sys.Dx)*.1
    #Q[2:,2:] = 0*Q[2:,2:] 
    Qf = np.eye(robot_sys.Dx)*100
    #Qf[2:,2:] = 0*Qf[2:,2:] 
    R = np.eye(robot_sys.Du)*1.
    
costs = []
for i in range(T):
    if i <= T_waypoint:
        cur_ref = x_waypoint.copy()
    elif i <= T_waypoint2:
         cur_ref = x_waypoint2.copy()        
    else:
        cur_ref = x_ref.copy()
    if i == T_waypoint or i == T_waypoint2: 
        Q_cur = 0.1*Qf.copy()
        Q_cur[3:6,3:6] *= 0
        Q_cur[9:,9:] *= 0
    else:
        Q_cur = Q.copy()
    runningStateCost = CostModelQuadratic(robot_sys, Q_cur, x_ref = cur_ref)
    runningControlCost = CostModelQuadratic(robot_sys, None, R)
    if use_collision:
        runningCost = CostModelSum(robot_sys, [runningStateCost, runningControlCost, cost_collision])
    else:
        runningCost = CostModelSum(robot_sys, [runningStateCost, runningControlCost])        
    costs += [runningCost]

#### Setup crocoddyl model
rmodels = []
for i in range(T):
    state = crocoddyl.StateVector(robot_sys.Dx)
    rmodel = ActionModelRobot(state, robot_sys.Du)
    rmodel.init_robot_sys(robot_sys, nr = robot_sys.Dx)
    rmodel.set_cost(costs[i])
    rmodels += [rmodel]

terminalStateCost = CostModelQuadratic(robot_sys,Qf, x_ref = x_ref)
terminalControlCost = CostModelQuadratic(robot_sys, None,R)
if use_collision:
    terminalCost = CostModelSum(robot_sys, [terminalStateCost, terminalControlCost, cost_collision])
else:
    terminalCost = CostModelSum(robot_sys, [terminalStateCost, terminalControlCost])

rmodel_T = ActionModelRobot(state, robot_sys.Du)
rmodel_T.init_robot_sys(robot_sys, nr = robot_sys.Dx)
rmodel_T.set_cost(terminalCost)

#### Setup problem & initial guess

In [101]:
#initial state
if sys_name == 'quadcopter':
    x0 = np.array([ 0.,  0., 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])
    us = np.zeros((T, robot_sys.Du))
elif sys_name == 'bicopter':
    x0 = np.zeros(6)
    u_shift = 9.8*robot_sys.m/2
    us = np.ones((T, robot_sys.Du))*u_shift

robot_sys.set_init_state(x0)
xs = robot_sys.rollout(us)
problem = crocoddyl.ShootingProblem(x0, rmodels, rmodel_T)

#### Solve

In [102]:
ddp = crocoddyl.SolverFDDP(problem)

ddp.th_grad = 1e-6
ddp.th_stop = 1e-6
ddp.setCallbacks([crocoddyl.CallbackVerbose()])

ddp.solve(list(xs[:,:,None]), list(us[:,:,None]), maxiter=1000)

xs_ref, us_ref = np.array(ddp.xs), np.array(ddp.us)

#### Visualize the path by multiple robots

In [103]:
if sys_name == 'quadcopter':
    for t in range(T):
        pos = xs_ref[t, :3]
        ori = euler2quat(xs_ref[t,6:9], 'rzyz')
        p.resetBasePositionAndOrientation(robotIds[t], pos, ori)

#### For report: show the trajectory

#### Copy the data from ddp to ilqr solutions

In [104]:
ilqr = ILQR_Standard(robot_sys)
ilqr = get_ilqr_from_ddp(ddp,ilqr)

#### Use Least Square to compute the distribution

In [105]:
from scipy.stats import multivariate_normal

In [106]:
n_samples = 200
_ = ilqr.compute_du_LS()
sample_dxs, sample_dus = ilqr.sample_du(n_samples, allow_singular=False)

sample_dxs = np.array(sample_dxs).reshape(n_samples, T+1, -1)

sampling


#### Plot the cross-section distribution

#### Plot for each dimension

In [107]:
cost_ori = ddp.problem.calc(list(xs_ref), list(us_ref))

In [108]:
xs_sample = xs_ref #+ sample_dxs[np.random.randint(n_samples)]

for t in range(T):
    pos = xs_sample[t, :3]
    ori = euler2quat(xs_ref[t,6:9], 'rzyz')
    p.resetBasePositionAndOrientation(robotIds[t], pos, ori)

# 2. Tracking the distribution with various strategies

## Defining the short time horizon ilqr

In [109]:
#### Setting cost
costs = []
for i in range(T_short+1):
    runningStateCostShort = CostModelQuadratic(robot_sys, Qf, x_ref = x_ref.copy())
    runningControlCostShort = CostModelQuadratic(robot_sys, None, R)
    if use_collision:
        runningCostShort = CostModelSum(robot_sys, [runningStateCostShort, runningControlCostShort, cost_collision])
    else:
        runningCostShort = CostModelSum(robot_sys, [runningStateCostShort, runningControlCostShort])        
    costs += [runningCostShort]

#### Setup crocoddyl model
rmodels = []
for i in range(T_short+1):
    stateShort = crocoddyl.StateVector(robot_sys.Dx)
    rmodelShort = ActionModelRobot(stateShort, robot_sys.Du)
    rmodelShort.init_robot_sys(robot_sys, nr = robot_sys.Dx)
    rmodelShort.set_cost(costs[i])
    rmodels += [rmodelShort]
    
reg_marg = 1e-2
reg_cond = 1e-2

#### Setup problem & initial guess

In [110]:
x0 = xs_ref[0]
robot_sys.set_init_state(x0)
xs = robot_sys.rollout(us)
problemShort = crocoddyl.ShootingProblem(x0, rmodels[:-1], rmodels[-1])
ddpShort = crocoddyl.SolverFDDP(problemShort)

#### Define the noises

In [203]:
if sys_name == 'quadcopter':
    noise_amp = 2.
    noise_v = normalize(np.random.rand(robot_sys.Dx/2)-0.5)*noise_amp
    noise = np.concatenate([np.zeros(3), noise_v[:3], np.zeros(3), noise_v[3:]])
    #noise = np.zeros(12)
    #noise[10] = -.5
    start_t = 65
    end_t = 67
elif sys_name == 'bicopter':
    noise[0:3] *= 0.
    start_t = 20
    end_t = 22
     
    
#set the timing of the noise
noises = np.zeros((T, robot_sys.Dx))
for t in range(start_t, end_t):
    noises[t] = noise
print(noise)

[ 0.     0.     0.     1.126  1.173 -0.758  0.     0.     0.     0.2
  0.354  0.785]


### 2.1. MPC Strategy: following the mean

In [204]:
Qs_mean = np.tile(Qf, (T+T_short+1,1,1))

#initialise x
x = x0.copy()

#set the output containers
xs_mean = [x]
us_mean = []
x_ref_mean_set = [] #ref at each iter
x_output_mean_set = [] #output of ddp at each iter.

#set initial guess
xs_init = xs_ref[:T_short+1]
us_init = us_ref[:T_short]
xs_init[0] = x.copy()

for t in range(T):
    print(t)
    
    #obtain and set the reference traj
    x_ref = subsample(xs_ref[t:t+T_short+1], T_short+1)
    set_ref(ddpShort, x_ref)
    Qs_ref = Qs_mean[t:t+T_short+1]
    set_Qref(ddpShort, Qs_ref)
    #modify the ddp initial point 
    ddpShort.problem.x0 = x
    ddpShort.x0 = x
    
    #solve the ddp
    ddpShort.solve(list(xs_init), list(us_init), n_iter_short)
    clear_output()
    
    #take the first control and execute (with noise)
    u = ddpShort.us[0]
    x = robot_sys.step(x, u) + noises[t]

    #store the data
    xs_mean += [x]
    us_mean += [u]
    x_ref_mean_set += [x_ref]
    x_output_mean_set += [np.array(ddpShort.xs)]
    
    #initialising the next step with the previous solutions
    xs_init = subsample(np.array(ddpShort.xs[1:]), T_short+1)
    us_init = subsample(np.array(ddpShort.us), T_short)
    xs_init[0] = x.copy()

xs_mean = np.array(xs_mean)
us_mean = np.array(us_mean)
cost_mean = ddp.problem.calc(list(xs_mean), list(us_mean))
print('The cost of tracking the mean is {}'.format(cost_mean))

The cost of tracking the mean is 1554.81592888


### 2.2. MPC Strategy: ILQR Feedback

In [205]:
#initialise x
x = x0.copy()

#setup the output container
xs_res_feed = [x]
us_res_feed = []

for t in range(T):
    print(t)
    u = ddp.us[t] - ddp.K[t].dot(x - ddp.xs[t])
    x = robot_sys.step(x, u) + noises[t]
    xs_res_feed += [x]
    us_res_feed += [u]
    
xs_res_feed = np.array(xs_res_feed)
us_res_feed = np.array(us_res_feed)
cost_feed = ddp.problem.calc(list(xs_res_feed), list(us_res_feed))
clear_output()
print('The cost of ilqr feedback control is is {}'.format(cost_feed))

The cost of ilqr feedback control is is 1308.95864897


### 2.3. MPC Strategy: Tracking marginal distribution

#### Compute marginal distribution

In [206]:
_, Qs = extract_ref(xs_ref[1:], ilqr.Sigma_x[robot_sys.Dx:, robot_sys.Dx:], robot_sys.Dx, T+T_short, xs_ref[0], reg_marg)

#### Tracking

In [207]:
#setup the initial state
x = x0.copy()

#setup the output containers
xs_res_dist = [x]
us_res_dist = []
x_ref_dist_set = []
x_output_dist_set = []

#setup the initial guess
xs_init = xs_ref[:T_short+1]
us_init = us_ref[:T_short]
xs_init[0] = x.copy()

for t in range(T):
    print(t)
    
    #extract the reference trajectory dist.
    x_ref = subsample(xs_ref[t:t+T_short+1], T_short+1)
    Qs_ref = Qs[t:t+T_short+1]
    set_ref(ddpShort, x_ref)
    set_Qref(ddpShort, Qs_ref)
    
    #modify the initial state of ddp
    ddpShort.problem.x0 = x
    ddpShort.x0 = x
    
    #solve
    ddpShort.solve(list(xs_init), list(us_init), n_iter_short)
    clear_output()
    
    #take the first control and execute
    u = ddpShort.us[0]
    x = robot_sys.step(x, u) + noises[t]

    #store the output
    xs_res_dist += [x]
    us_res_dist += [u]
    x_ref_dist_set += [x_ref]
    x_output_dist_set += [np.array(ddpShort.xs)]
    
    #set the initial guess from the previous solution
    xs_init = subsample(np.array(ddpShort.xs[1:]), T_short+1)
    us_init = subsample(np.array(ddpShort.us), T_short)
    xs_init[0] = x.copy()

xs_res_dist = np.array(xs_res_dist)
us_res_dist = np.array(us_res_dist)
cost_dist = ddp.problem.calc(list(xs_res_dist), list(us_res_dist))
clear_output()
print('The cost of tracking the marginal distribution is {}'.format(cost_dist))

The cost of tracking the marginal distribution is 473.347641407


### 2.4. MPC Strategy:  tracking conditional distribution

### Setup the trajectory distribution

In [208]:
#the full trajectory distribution
x_dist = Gaussian()
x_dist.D = robot_sys.Dx*(T+1+T_short)
x_dist.mu = ilqr.xs.flatten()
x_dist.sigma = np.zeros((ilqr.Sigma_x.shape[0]+T_short*robot_sys.Dx, ilqr.Sigma_x.shape[1]+T_short*robot_sys.Dx))
x_dist.sigma[:ilqr.Sigma_x.shape[0],:ilqr.Sigma_x.shape[0]] = ilqr.Sigma_x + 1e-6*np.eye(ilqr.Sigma_x.shape[0])
for t in range(T+1, T+1+T_short):
    x_dist.sigma[t*robot_sys.Dx:(t+1)*robot_sys.Dx, t*robot_sys.Dx:(t+1)*robot_sys.Dx] = x_dist.sigma[T*robot_sys.Dx:(T+1)*robot_sys.Dx, T*robot_sys.Dx:(T+1)*robot_sys.Dx]
    x_dist.mu = np.concatenate([x_dist.mu, xs_ref[-1]])
    #the conditional distribution along the time horizon T_short
x_dist_short = Gaussian()
x_dist_short.D = robot_sys.Dx*(T_short+1)

#### Tracking

In [209]:
#set initial state
x = x0.copy()

#set the output container
xs_res_cond = [x]
us_res_cond = []
x_ref_cond_set = []
x_output_cond_set = []

#set the initial guess
xs_init = xs_ref[:T_short+1]
us_init = us_ref[:T_short]
xs_init[0] = x.copy()

for t in range(T):
    print(t)
    
    mu,sigma = x_dist.get_marginal(slice(t*robot_sys.Dx, (t+1+T_short)*robot_sys.Dx))
    x_dist_short.mu = mu
    x_dist_short.sigma = sigma

    #get conditional distribution
    mu,sigma = x_dist_short.condition(x, slice(0,robot_sys.Dx), slice(robot_sys.Dx, x_dist.mu.shape[0]))
    x_dist_short.mu = mu
    x_dist_short.sigma = sigma

    #obtain references
    x_ref, Qs_ref = extract_ref(x_dist_short.mu, x_dist_short.sigma, robot_sys.Dx, T_short, x, reg = 1e-2)

    #set the reference
    set_ref(ddpShort, x_ref)
    set_Qref(ddpShort, 1*Qs_ref)
        
    #modify ddp initial state
    ddpShort.problem.x0 = x
    ddpShort.x0 = x
    
    #solve
    ddpShort.solve(list(xs_init), list(us_init), n_iter_short)
    clear_output()
    
    #take the first control and execute
    u = ddpShort.us[0]
    x = robot_sys.step(x, u) + noises[t]
    
    #store the output into the containers
    xs_res_cond += [x]
    us_res_cond += [u]
    x_ref_cond_set += [x_ref]
    x_output_cond_set += [np.array(ddpShort.xs)]

    #set the initial guess
    xs_init = subsample(np.array(ddpShort.xs[1:]), T_short+1)
    us_init = subsample(np.array(ddpShort.us), T_short)
    xs_init[0] = x.copy()


xs_res_cond = np.array(xs_res_cond)
us_res_cond = np.array(us_res_cond)
cost_cond = ddp.problem.calc(list(xs_res_cond), list(us_res_cond))
clear_output()
print('The cost of tracking the conditional distribution is {}'.format(cost_cond))

The cost of tracking the conditional distribution is 393.443309818


#### Comparison

In [210]:
print('Feedback: {}, Mean: {}, Dist: {}, Cond: {}'.format(cost_feed, cost_mean, cost_dist, cost_cond))

Feedback: 1308.95864897, Mean: 1554.81592888, Dist: 473.347641407, Cond: 393.443309818


In [211]:
print('cost_state, cost_control, cost_col,  cost_goal')
cost_state, cost_control, cost_col, cost_goal = calc_detail_cost(xs_res_feed, us_res_feed, ddp)
print('feed', cost_state, cost_control, np.sum(cost_col), cost_goal)
cost_state, cost_control, cost_col, cost_goal = calc_detail_cost(xs_mean, us_mean, ddp)
print('mean', cost_state, cost_control, np.sum(cost_col), cost_goal)
cost_state, cost_control, cost_col, cost_goal = calc_detail_cost(xs_res_dist, us_res_dist, ddp)
print('dist', cost_state, cost_control, np.sum(cost_col), cost_goal)
cost_state, cost_control, cost_col, cost_goal = calc_detail_cost(xs_res_cond, us_res_cond, ddp)
print('cond', cost_state, cost_control, np.sum(cost_col), cost_goal)


cost_state, cost_control, cost_col,  cost_goal
('feed', 292.93098231802, 142.6986442417209, 581.0264490369444, 297.98787274913616)
('mean', 59.93057123066875, 1246.6494552661738, 193.651850632743, 54.61877714234526)
('dist', 61.024979009859436, 336.3106471063261, 46.375208891289184, 29.67581384637001)
('cond', 222.3837339447822, 151.2932380983781, 5.2000202833420825, 15.279028385283253)


In [200]:
xs_ref_now = x_ref_cond_set[end_t]
for t in range(T_short):
    pos = xs_ref_now[t, :3]
    ori = euler2quat(xs_ref[t,6:9], 'rzyz')
    p.resetBasePositionAndOrientation(robotIdRefs[t], pos, ori)

xs_ref_now = x_ref_mean_set[end_t]
for t in range(T_short):
    pos = xs_ref_now[t, :3]
    ori = euler2quat(xs_ref[t,6:9], 'rzyz')
    p.resetBasePositionAndOrientation(robotIdRefMeans[t], pos, ori)

In [201]:
robot_sys.vis_traj(robotId, xs_res_feed, camDist = 4, camPitch = -50, camYaw = 270)

In [97]:
robot_sys.vis_traj(robotId, xs_mean, camDist = 4, camPitch = -50, camYaw = 270)

In [161]:
robot_sys.vis_traj(robotId, xs_res_dist, camDist = 4, camPitch = -50, camYaw = 270)

In [180]:
robot_sys.vis_traj(robotId, xs_res_cond, camDist = 4, camPitch = -50, camYaw = 270, changeCamera = True)

#### The reference trajectory

In [95]:
%timeit -n 3 -r 3 ddpShort.solve(list(xs_init), list(us_init), 1)

3 loops, best of 3: 38.9 ms per loop


In [96]:
%timeit -n 3 -r 3 ddp.solve(list(xs_ref), list(us_ref), 1)

3 loops, best of 3: 179 ms per loop


***

# Batch comparison

In [260]:
import time
tic = time.time()
n_test = 20

to_build_from_zero = False
noise_sizes = ['small', 'medium', 'large']
method_names = ['feed', 'mean', 'marg', 'cond']
field_names = ['xs', 'us', 'x_ref_set', 'x_output_set', 'cost', 'noise', 
               'cost_state', 'cost_col', 'cost_goal', 'cost_control']
if to_build_from_zero:
    data = {'small':dict(), 'medium':dict(), 'large':dict()}
    for noise_size in noise_sizes:
        for method_name in method_names:
            data[noise_size][method_name] = dict()
            for field_name in field_names:
                data[noise_size][method_name][field_name] = []
else:
    data = np.load('data/ilqr_'+sys_name+'4.npy', allow_pickle = True)[()]
    for noise_size in noise_sizes:
        for method_name in method_names:
            for field_name in field_names:
                data[noise_size][method_name][field_name] = list(data[noise_size][method_name][field_name])

In [262]:
for noise_size in noise_sizes[:]:
#     if noise_size == 'medium':
#         start_k = 16
#     else:
#         start_k = 0
        
    for k in range(start_k, n_test):
        #### Define the noises
        if sys_name == 'quadcopter':           
            if noise_size == 'large':
                noise_amp = 1.2 #3.
            elif noise_size == 'medium':
                noise_amp = 0.7#1.5
            elif noise_size == 'small':
                noise_amp = 0.2#0.5
            noise_v = normalize(np.random.rand(robot_sys.Dx/2)-0.5)*noise_amp
            noise = np.concatenate([np.zeros(3), noise_v[:3], np.zeros(3), noise_v[3:]])
            #noise = np.zeros(12)
            #noise[10] = -.5
            start_t = 65
            end_t = 67
        elif sys_name == 'bicopter':
            if noise_size == 'large':
                noise_amp = 10.
            elif noise_size == 'medium':
                noise_amp = 5.
            elif noise_size == 'small':
                noise_amp = 1
            noise = np.random.rand(robot_sys.Dx)*noise_amp
            noise[0:3] *= 0.
            start_t = 20
            end_t = 22

        #set the timing of the noise
        noises = np.zeros((T, robot_sys.Dx))
        for t in range(start_t, end_t):
            noises[t] = noise

        ### 2.1. MPC Strategy: following the mean
        Qs_mean = np.tile(Qf, (T+T_short+1,1,1))

        #initialise x
        x = x0.copy()

        #set the output containers
        xs_mean = [x]
        us_mean = []
        x_ref_mean_set = [] #ref at each iter
        x_output_mean_set = [] #output of ddp at each iter.

        #set initial guess
        xs_init = xs_ref[:T_short+1]
        us_init = us_ref[:T_short]
        xs_init[0] = x.copy()

        for t in range(T):
            print(t)

            #obtain and set the reference traj
            x_ref = subsample(xs_ref[t:t+T_short+1], T_short+1)
            set_ref(ddpShort, x_ref)
            Qs_ref = Qs_mean[t:t+T_short+1]
            set_Qref(ddpShort, Qs_ref)
            #modify the ddp initial point 
            ddpShort.problem.x0 = x
            ddpShort.x0 = x

            #solve the ddp
            ddpShort.solve(list(xs_init), list(us_init), n_iter_short)
            clear_output()

            #take the first control and execute (with noise)
            u = ddpShort.us[0]
            x = robot_sys.step(x, u) + noises[t]

            #store the data
            xs_mean += [x]
            us_mean += [u]
            x_ref_mean_set += [x_ref]
            x_output_mean_set += [np.array(ddpShort.xs)]

            #initialising the next step with the previous solutions
            xs_init = subsample(np.array(ddpShort.xs[1:]), T_short+1)
            us_init = subsample(np.array(ddpShort.us), T_short)
            xs_init[0] = x.copy()

        xs_mean = np.array(xs_mean)
        us_mean = np.array(us_mean)
        cost_mean = ddp.problem.calc(list(xs_mean), list(us_mean))
        print('The cost of tracking the mean is {}'.format(cost_mean))


        ### 2.2. MPC Strategy: ILQR Feedback

        #initialise x
        x = x0.copy()

        #setup the output container
        xs_res_feed = [x]
        us_res_feed = []

        for t in range(T):
            print(t)
            u = ddp.us[t] - ddp.K[t].dot(x - ddp.xs[t])
            x = robot_sys.step(x, u) + noises[t]
            xs_res_feed += [x]
            us_res_feed += [u]

        xs_res_feed = np.array(xs_res_feed)
        us_res_feed = np.array(us_res_feed)
        cost_feed = ddp.problem.calc(list(xs_res_feed), list(us_res_feed))
        clear_output()
        print('The cost of ilqr feedback control is is {}'.format(cost_feed))


        ### 2.3. MPC Strategy: Tracking marginal distribution

        #### Compute marginal distribution
        _, Qs = extract_ref(xs_ref[1:], ilqr.Sigma_x[robot_sys.Dx:, robot_sys.Dx:], 
                            robot_sys.Dx, T+T_short, xs_ref[0], reg_marg)

        #### Tracking

        #setup the initial state
        x = x0.copy()

        #setup the output containers
        xs_res_dist = [x]
        us_res_dist = []
        x_ref_dist_set = []
        x_output_dist_set = []

        #setup the initial guess
        xs_init = xs_ref[:T_short+1]
        us_init = us_ref[:T_short]
        xs_init[0] = x.copy()

        for t in range(T):
            print(t)

            #extract the reference trajectory dist.
            x_ref = subsample(xs_ref[t:t+T_short+1], T_short+1)
            Qs_ref = Qs[t:t+T_short+1]
            set_ref(ddpShort, x_ref)
            set_Qref(ddpShort, Qs_ref)

            #modify the initial state of ddp
            ddpShort.problem.x0 = x
            ddpShort.x0 = x

            #solve
            ddpShort.solve(list(xs_init), list(us_init), n_iter_short)
            clear_output()

            #take the first control and execute
            u = ddpShort.us[0]
            x = robot_sys.step(x, u) + noises[t]

            #store the output
            xs_res_dist += [x]
            us_res_dist += [u]
            x_ref_dist_set += [x_ref]
            x_output_dist_set += [np.array(ddpShort.xs)]

            #set the initial guess from the previous solution
            xs_init = subsample(np.array(ddpShort.xs[1:]), T_short+1)
            us_init = subsample(np.array(ddpShort.us), T_short)
            xs_init[0] = x.copy()

        xs_res_dist = np.array(xs_res_dist)
        us_res_dist = np.array(us_res_dist)
        cost_dist = ddp.problem.calc(list(xs_res_dist), list(us_res_dist))
        clear_output()
        print('The cost of tracking the marginal distribution is {}'.format(cost_dist))


        ### 2.4. MPC Strategy:  tracking conditional distribution

        ### Setup the trajectory distribution

        #the full trajectory distribution
        x_dist = Gaussian()
        x_dist.D = robot_sys.Dx*(T+1+T_short)
        x_dist.mu = ilqr.xs.flatten()
        x_dist.sigma = np.zeros((ilqr.Sigma_x.shape[0]+T_short*robot_sys.Dx, ilqr.Sigma_x.shape[1]+T_short*robot_sys.Dx))
        x_dist.sigma[:ilqr.Sigma_x.shape[0],:ilqr.Sigma_x.shape[0]] = ilqr.Sigma_x + 1e-6*np.eye(ilqr.Sigma_x.shape[0])
        for t in range(T+1, T+1+T_short):
            x_dist.sigma[t*robot_sys.Dx:(t+1)*robot_sys.Dx, t*robot_sys.Dx:(t+1)*robot_sys.Dx] = x_dist.sigma[T*robot_sys.Dx:(T+1)*robot_sys.Dx, T*robot_sys.Dx:(T+1)*robot_sys.Dx]
            x_dist.mu = np.concatenate([x_dist.mu, xs_ref[-1]])
            #the conditional distribution along the time horizon T_short
        x_dist_short = Gaussian()
        x_dist_short.D = robot_sys.Dx*(T_short+1)

        #### Tracking

        #set initial state
        x = x0.copy()

        #set the output container
        xs_res_cond = [x]
        us_res_cond = []
        x_ref_cond_set = []
        x_output_cond_set = []

        #set the initial guess
        xs_init = xs_ref[:T_short+1]
        us_init = us_ref[:T_short]
        xs_init[0] = x.copy()

        for t in range(T):
            print(t)

            mu,sigma = x_dist.get_marginal(slice(t*robot_sys.Dx, (t+1+T_short)*robot_sys.Dx))
            x_dist_short.mu = mu
            x_dist_short.sigma = sigma

            #get conditional distribution
            mu,sigma = x_dist_short.condition(x, slice(0,robot_sys.Dx), slice(robot_sys.Dx, x_dist.mu.shape[0]))
            x_dist_short.mu = mu
            x_dist_short.sigma = sigma

            #obtain references
            x_ref, Qs_ref = extract_ref(x_dist_short.mu, x_dist_short.sigma, robot_sys.Dx, T_short, x, reg = 1e-2)

            #set the reference
            set_ref(ddpShort, x_ref)
            set_Qref(ddpShort, 1*Qs_ref)

            #modify ddp initial state
            ddpShort.problem.x0 = x
            ddpShort.x0 = x

            #solve
            ddpShort.solve(list(xs_init), list(us_init), n_iter_short)
            clear_output()

            #take the first control and execute
            u = ddpShort.us[0]
            x = robot_sys.step(x, u) + noises[t]

            #store the output into the containers
            xs_res_cond += [x]
            us_res_cond += [u]
            x_ref_cond_set += [x_ref]
            x_output_cond_set += [np.array(ddpShort.xs)]

            #set the initial guess
            xs_init = subsample(np.array(ddpShort.xs[1:]), T_short+1)
            us_init = subsample(np.array(ddpShort.us), T_short)
            xs_init[0] = x.copy()


        xs_res_cond = np.array(xs_res_cond)
        us_res_cond = np.array(us_res_cond)
        cost_cond = ddp.problem.calc(list(xs_res_cond), list(us_res_cond))
        clear_output()
        print('The cost of tracking the conditional distribution is {}'.format(cost_cond))


        #### Comparison
        print('Feedback: {}, Mean: {}, Dist: {}, Cond: {}'.format(cost_feed, cost_mean, cost_dist, cost_cond))


        ####store the data
        #data mean
        cost_state, cost_control, cost_col, cost_goal  = calc_detail_cost(xs_mean, us_mean, ddp)
        data[noise_size]['mean']['xs'] += [xs_mean]
        data[noise_size]['mean']['us'] += [us_mean]
        data[noise_size]['mean']['x_ref_set'] += [x_ref_mean_set]
        data[noise_size]['mean']['x_output_set'] += [x_output_mean_set]
        data[noise_size]['mean']['cost'] += [cost_mean]
        data[noise_size]['mean']['noise'] += [noise]
        data[noise_size]['mean']['cost_state'] += [cost_state]
        data[noise_size]['mean']['cost_control'] += [cost_control]
        data[noise_size]['mean']['cost_col'] += [cost_col]
        data[noise_size]['mean']['cost_goal'] += [cost_goal]

        #data feedback
        cost_state, cost_control, cost_col, cost_goal  = calc_detail_cost(xs_res_feed, us_res_feed, ddp)
        data[noise_size]['feed']['xs'] += [xs_res_feed]
        data[noise_size]['feed']['us'] += [us_res_feed]
        data[noise_size]['feed']['cost'] += [cost_feed]
        data[noise_size]['feed']['noise'] += [noise]
        data[noise_size]['feed']['cost_state'] += [cost_state]
        data[noise_size]['feed']['cost_control'] += [cost_control]
        data[noise_size]['feed']['cost_col'] += [cost_col]
        data[noise_size]['feed']['cost_goal'] += [cost_goal]


        #data margin
        cost_state, cost_control, cost_col, cost_goal  = calc_detail_cost(xs_res_dist, us_res_dist, ddp)
        data[noise_size]['marg']['xs'] += [xs_res_dist]
        data[noise_size]['marg']['us'] += [us_res_dist]
        data[noise_size]['marg']['x_ref_set'] += [x_ref_dist_set]
        data[noise_size]['marg']['x_output_set'] += [x_output_dist_set]
        data[noise_size]['marg']['cost'] += [cost_dist]
        data[noise_size]['marg']['noise'] += [noise]
        data[noise_size]['marg']['cost_state'] += [cost_state]
        data[noise_size]['marg']['cost_control'] += [cost_control]
        data[noise_size]['marg']['cost_col'] += [cost_col]
        data[noise_size]['marg']['cost_goal'] += [cost_goal]

        #data 
        cost_state, cost_control, cost_col, cost_goal  = calc_detail_cost(xs_res_cond, us_res_cond, ddp)
        data[noise_size]['cond']['xs'] += [xs_res_cond]
        data[noise_size]['cond']['us'] += [us_res_cond]
        data[noise_size]['cond']['x_ref_set'] += [x_ref_cond_set]
        data[noise_size]['cond']['x_output_set'] += [x_output_cond_set]
        data[noise_size]['cond']['cost'] += [cost_cond]
        data[noise_size]['cond']['noise'] += [noise]
        data[noise_size]['cond']['cost_state'] += [cost_state]
        data[noise_size]['cond']['cost_control'] += [cost_control]
        data[noise_size]['cond']['cost_col'] += [cost_col]
        data[noise_size]['cond']['cost_goal'] += [cost_goal]

        print('Feedback: {}, Mean: {}, Dist: {}, Cond: {}'.format(cost_feed, cost_mean, cost_dist, cost_cond))
        print('cost_state, cost_control, cost_col,  cost_goal')
        cost_state, cost_control, cost_col, cost_goal = calc_detail_cost(xs_res_feed, us_res_feed, ddp)
        print('feed', cost_state, cost_control, np.sum(cost_col), cost_goal)
        cost_state, cost_control, cost_col, cost_goal = calc_detail_cost(xs_mean, us_mean, ddp)
        print('mean', cost_state, cost_control, np.sum(cost_col), cost_goal)
        cost_state, cost_control, cost_col, cost_goal = calc_detail_cost(xs_res_dist, us_res_dist, ddp)
        print('dist', cost_state, cost_control, np.sum(cost_col), cost_goal)
        cost_state, cost_control, cost_col, cost_goal = calc_detail_cost(xs_res_cond, us_res_cond, ddp)
        print('cond', cost_state, cost_control, np.sum(cost_col), cost_goal)

The cost of tracking the conditional distribution is 259.551317225
Feedback: 317.559414353, Mean: 816.895756428, Dist: 378.677039505, Cond: 259.551317225
Feedback: 317.559414353, Mean: 816.895756428, Dist: 378.677039505, Cond: 259.551317225
cost_state, cost_control, cost_col,  cost_goal
('feed', 95.05186138321423, 219.27162777450243, 0.2132539313872136, 3.1012579159902316)
('mean', 51.77814639188214, 637.8956379585045, 108.19617056343658, 19.053716698651993)
('dist', 57.15565033192094, 298.96143359585005, 1.8602338425678533, 20.72720094829253)
('cond', 102.48885287783375, 152.3915498751388, 0.16825738079682284, 4.657849329598317)


In [263]:
for noise_size in noise_sizes:
    for method_name in method_names:
        for field_name in field_names:
            data[noise_size][method_name][field_name] = np.array(data[noise_size][method_name][field_name])
    

toc = time.time()
print('Computation takes {}'.format(toc-tic))

Computation takes 4497.92493701


In [264]:
np.save('data/ilqr_'+sys_name+'4.npy', data)

In [227]:
from playsound import playsound
 
playsound('data/phantom.mp3')